### 1、将要处理数据，常规标签，非常规标签，工商关联表放在同一文件夹下，名称规范
#### 该文件夹下子文件夹名不更改，文件名不更改
### 2、将代码工作路径调整到该文件夹
### 3、运行代码
### 4、结果输出到该文件夹下
#### 统计表，流水表

In [1]:
import numpy as np
import pandas as pd
import os
import re
import datetime

In [2]:
# 工作路径调整到工作路径
def path_1(path):
    os.chdir(path)
    print("当前工作路径:",os.getcwd())
    return os.listdir()
  
    
# 修改 path 工作路径    
path = r"E:\1-谢成\工作资料-在家办公\标准化测试文件夹"
# 打印工作路径 及 路径下文件
path_1(path)

当前工作路径: E:\1-谢成\工作资料-在家办公\标准化测试文件夹


['1-数据清洗.ipynb', 'lml数据清洗.xlsx', 'read.txt', '策略文件']

### 数据清洗
#### 标准字段

In [19]:
# 18个模板字段信息
stand_data  = pd.DataFrame(columns=["交易账号","交易卡号","交易户名",    "交易日期","交易时间","交易币种",
                                    "借贷标志","交易金额","交易余额",    "交易类型","交易发生地",
                              "对方账号","对方卡号","对方户名","对方银行", "摘要","备注","附言"])

In [20]:
stand_data.columns

Index(['交易账号', '交易卡号', '交易户名', '交易日期', '交易时间', '交易币种', '借贷标志', '交易金额', '交易余额',
       '交易类型', '交易发生地', '对方账号', '对方卡号', '对方户名', '对方银行', '摘要', '备注', '附言'],
      dtype='object')

In [21]:
# 数据要求excel（.xlsx文件），单个sheet
# 数据批量读取
# 数据读取后与 stand_data 进行拼接

In [23]:
# 读数据，只读 .xlsx
def read_xl():
    list_o = os.listdir()
    list_o_1 = []
    for i in list_o: 
        if ".xlsx" in i :       # 确保只读取 .xlsx 的 excel 文件
            data_o = pd.read_excel(i,dtype=np.str)
            list_o_1.append(data_o)
    return list_o_1

list_o_1 = read_xl()   # 导入后数据 list_o_1

In [24]:
#for i in list_o_1:
    #i.drop(columns=['查询反馈结果原因\t','交易证件号码\t','现金标志\t','渠道\t','日志号\t', '凭证种类\t', '凭证号\t', '交易柜员号\t',],inplace=True)
    #i.drop(columns=['MAC地址\t','对手身份证号\t',],inplace=True)
    #i.drop(columns=['传票号\t', 'IP地址\t', '对手交易余额\t','交易流水号\t','交易场所\t','交易是否成功\t',],inplace=True)
    #print(i.columns)

## 借贷标志0 1

In [26]:
# 清洗
list_o_2 = []
for i,data_i in enumerate(list_o_1):
    for col in data_i.columns:              # 空格，换行等清洗
        data_i[col] = data_i[col].str.strip().str.replace("'","").str.replace("(","（").str.replace(")","）")
    x1 = set(data_i.借贷标志.str.strip().unique())
    if x1 == {'进','出'}:
        list_o_2.append(data_i)
    elif x1 == {"0","1"}:             
        data_i.借贷标志.replace({"0":"出","1":"进"},inplace = True)   #替换1
        list_o_2.append(data_i)                
    elif x1 == {"C","D"}:
        data_i.借贷标志.replace({"C":"进","D":"出"},inplace = True)   # 替换2
        list_o_2.append(data_i)                   
    else :
        print("需要补充规则i：",x1)                         # 规则未修改的数据是不会做数据合并的
        print(i)

list_o_2.append(stand_data)
# 合并数据
data_x = pd.concat(list_o_2,ignore_index=True)

需要补充规则i： {'', '进', '出'}
0


In [7]:
# 注：数据空值以浮点数形式存在是无法进行清洗的

In [8]:
# 处理方式，数据分离为日期，时间两列，不在代码清洗中进行拆分
# 存在情况是 只有日期或者日期时间均有两种模式，拆分条件不好判断如（2019-07-02,20190702），拆分前种类也较多  
# 后续跟进 以字符长度进行自动处理

# 日期及时间 格式如下
# 2019/12/21 ...

In [9]:
# 日期转换  三种日期格式
def date_1(x):
    if " " in str(x):
        x = str(x).split(" ")[0]
        x = datetime.datetime.strptime(x,"%Y-%m-%d").date()
        return x
    
    elif type(x)!=np.float:
        try :
            x = datetime.datetime.strptime(x,"%Y-%m-%d").date()
        except:
            try :
                x = datetime.datetime.strptime(x,"%Y/%m/%d").date()
            except:
                try :
                    x = datetime.datetime.strptime(x,"%Y%m%d").date()
                except:
                    print(x)
        return x
    
# 时间转换  （092456,可检测) (92456,不可检测，或转换错误)
def time_1(x):
    if type(x)!=np.float:
        try :
            x = datetime.datetime.strptime(x,"%H:%M:%S").time()
        except:
            try :
                x = datetime.datetime.strptime(x,"%H.%M.%S").time()
            except:
                try :
                    if len(x)==6:                                         # 需要满足数据是时间的6位长度
                        x = datetime.datetime.strptime(x,"%H%M%S").time()
                except:
                    try:
                        x = datetime.datetime.strptime(x,"%H：%M：%S").time()
                    except:
                        print(x)
        return x    

In [10]:
# 金额数值化
def zifu(data = data_x,str_1 =","):  # str_1 需要清洗的字符,默认是 , 逗号
    len_1 = len(data_x[data_x.交易金额.str.contains(str_1)])
    print("包含特殊符号的交易数量：",len_1)
    
    if 1>=0 :
        data_x["交易金额"] = data_x.交易金额.map(lambda x :str(x).replace(str_1,""))
        data_x["交易余额"] = data_x.交易余额.map(lambda x :str(x).replace(str_1,""))
    data_x["交易金额"] = pd.to_numeric(data_x.交易金额,errors="raise")
    data_x["交易余额"] = pd.to_numeric(data_x.交易余额,errors="ignore")   
    # 转换不成功会报错
    # 错误情况 1、已经是数值的二次处理;2、其他特殊字符，无法转换
    return data_x    

In [11]:
# 清洗
data_x["交易日期"] = data_x.交易日期.map(lambda x:date_1(x))
data_x["交易时间"] = data_x.交易时间.map(lambda x:time_1(x))

In [12]:
data_xie = zifu(data=data_x,str_1=",")    # 传入数据表 data ， 金额中的其他特殊字符 str_1

包含特殊符号的交易数量： 0


### 缺失对方账号，对方户名填补
##### 1、先补充账号，账号均不为空，再补充对方户名
##### 2、对方户名缺失的补充上对方账号

In [13]:
data_lgh_1 = data_xie.copy()  # copy数据，data_xie,清洗好的数据可以直接写出或再次利用

In [14]:
# 辅助列"四合一"  合并字符串
data_lgh_1["四合一"] = (data_lgh_1.交易类型.map(str)+"_"+data_lgh_1.摘要.map(str)+"_"+
                     data_lgh_1.备注.map(str)+"_"+data_lgh_1.附言.map(str))

In [15]:
# 对方账号
def tb_a_j(x,y):       # x 借贷标志；y 四合一
    
    if x == "进":
        parten1 = re.compile("利息|结息|利息存入|储蓄结息|利息资本化结息入账|支付利息")
        parten2 = re.compile("产品本金|聚财到期")
        parten3 = re.compile("税后利息")
        parten4 = re.compile("消费退货")
        parten5 = re.compile("现金|ATM存款|现存|现金存入")
        
        if re.search(parten5,y) !=None :
            z = "现金存入"
        elif re.search(parten4,y) != None :
            z = "消费退款"
        elif re.search(parten3,y) != None :
            z = "理财利息"
        elif re.search(parten2,y) != None :
            z = "银行理财汇款"
        elif re.search(parten1,y) != None :
            z = "银行利息"        
        else :
            z = "未知流入"
    else:         #默认其他所有是 出
        parten1 = re.compile("冲正")
        parten2 = re.compile("支付手续费|账户管理费|手续费|个人账户管理费|跨行费")
        parten3 = re.compile("理财产品扣款|基金认购|理财产品扣划")
        parten4 = re.compile("消费|酒店|鞋城|男装|宾馆|阿玛尼|购物|烧烤|百货|路易威登|餐饮|娱乐|服饰|医院|超市| \
                             航空|加油站|车行|二手车|旅行社|保险|电视|火锅|玩具|蛋糕|石油|天然气|珠宝|房地产|汽车| \
                             电器|通讯器材|商场|眼镜|商店|万达|饭店|美团|饿了么|俱乐部|餐厅|家居")
        parten5 = re.compile("ATM取款|卡取|网络ATM取款|现金支取|跨行自助取款|ATM取现|柜台取现|银联ATM取款")
        
        if re.search(parten5,y) !=None :
            z = "现金取出"
        elif re.search(parten4,y) != None :
            z = "消费"
        elif re.search(parten3,y) != None :
            z = "购买理财"
        elif re.search(parten2,y) != None :
            z = "银行手续费"
        elif re.search(parten1,y) != None :
            z = "冲正"
        else :
            z = "未知流出"
    return z

In [16]:
# 对方户名
def tb_a_h(x):       # x 账号
    parten1 = re.compile("银行利息|银行理财回款|理财利息|消费退款|现金存入|冲正|银行手续费|购买理财|消费|现金取出")
    if re.search(parten1,x) != None:
        z = re.findall(parten1,x)[0]  
    else:       
        z = "未知对手"
    return z

In [17]:
# 对方账号
data_1111 = data_lgh_1[(data_lgh_1.对方账号.isnull())|(data_lgh_1.对方账号 == '')]
data_lgh_1.loc[(data_lgh_1.对方账号.isnull())|(data_lgh_1.对方账号 == ''),"对方账号"] = list(
                                                    map(lambda x,y:tb_a_j(x,y),data_1111.借贷标志,data_1111.四合一))

In [18]:
# 对方户名 填充
data_1112 = data_lgh_1[(data_lgh_1.对方户名.isnull())|(data_lgh_1.对方户名 == '')]
data_lgh_1.loc[(data_lgh_1.对方户名.isnull())|(data_lgh_1.对方户名 == ''),"对方户名"] = list(map(lambda x:tb_a_h(x),data_1112.对方账号))

### 冲正模型

In [19]:
print(type(data_lgh_1.交易日期[0])) # 看交易日期 时间格式
print(type(data_lgh_1.交易时间[0])) # 看交易时间 时间格式
# 如果不满足需要再处理时间

<class 'datetime.date'>
<class 'datetime.time'>


In [20]:
# 只提取包含交易日期交易时间的数据进行处理
data_lgh_on = data_lgh_1[data_lgh_1.交易时间.notnull()] # 数据拆分
data_lgh_no_time = data_lgh_1[data_lgh_1.交易时间.isnull()]

In [21]:
# 数据排序,对有交易时间数据操作
data_lgh_on.sort_values(by=["交易账号","交易卡号","交易户名","交易日期","交易时间"],
                       ascending=True,inplace=True)  # ascending=True 升序排列
data_lgh_on.reset_index(drop=True,inplace=True)      # 数据排序后重新建立索引

In [22]:
# 四合一已经定义为字符
# 锁定冲正 数据 位置
list_cz_a = data_lgh_on[data_lgh_on.四合一.str.contains("冲正")].index
print("含冲正字样数据数量：",len(list_cz_a)) 

含冲正字样数据数量： 85


In [23]:
# 会清洗掉一部分，剩下的会标记出来，可以手动清洗
data_lgh_on["有误账款标记"] =  np.nan
data_lgh_on.loc[list_cz_a,"有误账款标记"] = "手动调整"
for cz in list_cz_a:
    cz_0 = data_lgh_on.loc[cz,"交易金额"] + data_lgh_on.iloc[cz-1,"交易金额"]
    if int(cz_0)==0:
        data_lgh_on.drop(index = [cz,cz-1],inplace=True)

In [24]:
## 冲正结束，数据合并
data_l = pd.concat([data_lgh_on,data_lgh_no_time],sort=False)

### 基础统计
#### 交易户名，交易账号，交易卡号，进出账交易金额，交易笔数，净流入金额，交易起止日期

In [25]:
data_d = data_l.copy()    # copy 同上  # 基于冲正剔除后数据整理

In [26]:
#1 金额,笔数统计
tj_1 = pd.pivot_table(data_d,index=["交易户名","交易账号","交易卡号"],
              columns=["借贷标志"],values=["交易金额"],aggfunc=[np.sum,"count"])
tj_1.columns = ["总出账金额(元)","总进账金额(元)","count_c","count_j"]

In [51]:
#2 最早最晚交易日期统计统计
data_d.sort_values(by=["交易账号","交易卡号","交易户名","交易日期","交易时间"],
                       ascending=True,inplace=True)                   # 升序
zz_first = data_d.drop_duplicates(["交易账号","交易卡号","交易户名"],keep="first")[["交易账号","交易卡号","交易户名","交易日期"]]
zw_last = data_d.drop_duplicates(["交易账号","交易卡号","交易户名"],keep="last")[
    ["交易账号","交易卡号","交易户名","交易日期","交易时间","交易余额"]]


In [52]:
tj_2 = tj_1.merge(zz_first,how="left",left_on=["交易账号","交易卡号","交易户名"],
          right_on=["交易账号","交易卡号","交易户名"],suffixes=("_开始","_截止"))
tj_3 = tj_2.merge(zw_last,how="left",left_on=["交易账号","交易卡号","交易户名"],
          right_on=["交易账号","交易卡号","交易户名"],suffixes=("_开始","_截止"))

In [53]:
tj_3["净流入金额(元)"] = tj_3["总进账金额(元)"] - tj_3["总出账金额(元)"]
tj_3["总交易笔数"] = tj_3["count_c"] + tj_3["count_j"]
tj_3.drop(columns=["count_c","count_j"],inplace=True)
# 无交易时间整理
tj_3.loc[tj_3.交易时间.isnull(),"交易余额"] = "无交易时间，无法判断"
tj_3.drop(columns=["交易时间"],inplace=True)

In [54]:
tj_3.sort_values(by=["总进账金额(元)"],ascending=False,inplace=True)

### 常规标签

In [32]:
##   常规词库
rule_ci = pd.read_excel(r"策略文件\2020_4_1常规标签修正.xlsx",dtype=np.str,header=[0,1,2])

In [33]:
data_lz = data_l.copy()

In [34]:
def biaoqian(df=rule_ci,data = data_lz):
    data["字面用途标签"] = np.nan       # 标签
    for col_name in df.columns:
        print(col_name)
        guize =  "|".join(list(filter(lambda x:str(x)!="nan",df[col_name])))  # 规则
        parten = re.compile(guize)
        print(guize,"\n")
        data.loc[(data.字面用途标签.isnull())&
                 (data[col_name[2]].map(lambda x :re.search(parten,x)!=None)) &
                 (data.借贷标志.map(lambda x:x in col_name[0])) ,"字面用途标签"]   = col_name[1]
    data.loc[(data.交易日期.map(lambda x:x.day)) & (data.四合一.str.contains("利息")),"字面用途标签"] = "银行利息"
    
    return data

### 非常规标签

In [35]:
# 非常规词库
un_rule_ci = pd.read_excel(r"策略文件\非常规自定义策略.xlsx",dtype = np.str,header=[0,1,2,3,4])
un_rule_ci.drop(columns=[('借贷标志', '特征字段', '特征标签', '最终身份标签', '最终用途标签')],inplace = True)
un_rule_ci.head()

进                    出      进                      出  \
                  对方账号                 对方账号    四合一  对方户名 交易金额   四合一   四合一   
                   投资款                  返利款    购房款  营业收入 营业收入 疑似投资款   返利款   
                   投资人                  投资人    投资人  住宿客户 住宿客户 疑似投资人   投资人   
                   投资款                  返利款    购房款  营业收入 营业收入 疑似投资款   返利款   
0  6217982610006757175  6217982610006757175    购房款   旅行社  300   拉卡拉    返本   
1  6222022703000096674  6222022703000096674  购房预定款  航空公司  NaN   NaN    返息   
2  6212260505009320654  6212260505009320654     房款   NaN  NaN   NaN  返还本息   
3  6228430899015459619  6228430899015459619    NaN   NaN  NaN   NaN   NaN   
4  6217001020006044698  6217001020006044698    NaN   NaN  NaN   NaN   NaN   

                                进     出  
         对方户名    四合一   对方户名  对方户名  对方户名  
    购车款   购车款   运营支出   运营支出 疑似投资款 疑似返利款  
   重点关注  重点关注   企业运营   企业运营 疑似投资人 疑似投资人  
    购车款   购车款    运营款    运营款 疑似投资款 疑似投资款  
0     车     车    洗条厂    洗条厂   彭建湘   彭建湘  
1  销售汽车  销售汽车   酒店用品   酒店用品    唐燕    唐燕  
2  汽车销售  汽车销售    活动款    活动款   沈爱荣   沈爱荣  
3   车预定   车预定  活动奖励款  活动奖励款    方霞    方霞  
4   NaN   NaN    资料费    资料费    田静    田静

In [36]:
def biaoqian_2(df=un_rule_ci,data = None):
    
    data["非常规标签"] = np.nan       # 标签
    data["最终对手标签"] = np.nan
    data["最终用途标签"] = np.nan
    if len(df)==0:
        return data
    else:
        for col_name in df.columns:
            print(col_name)
            # 1、数据拼接
            if ((col_name[1]=="对方账号") | (col_name[1] =="对方户名")):     
                ds_info = df[[col_name]].drop_duplicates()
                ds_info.columns = [col_name[1]+"_1"]
                
                data = data.merge(ds_info,how="left",left_on=col_name[1],right_on=ds_info.columns[0])
                data.loc[
                    (data.非常规标签.isnull()) & (data.iloc[:,-1].notnull()) & 
                    (data.借贷标志.map(lambda x:x in col_name[0])),
                    "非常规标签"] = col_name[2]
                data.drop(columns=data.columns[-1],inplace=True)
                
                data.loc[data.非常规标签==col_name[2],"最终对手标签"] = col_name[3]
                data.loc[data.非常规标签==col_name[2],"最终用途标签"] = col_name[4]
            # 2、特殊交易金额计算
            elif col_name[1] == "交易金额":
                num_list = list(filter(lambda x:str(x)!="nan",df[col_name])) #
                for  num in num_list:
                    data.loc[
                        (data.非常规标签.isnull()) & 
                        (data.交易金额.map(lambda x:x==np.float(num))) & 
                        (data.借贷标志.map(lambda x:x in col_name[0])),
                        "非常规标签"] = col_name[2]
                    
                    data.loc[data.非常规标签==col_name[2],"最终对手标签"] = col_name[3]
                    data.loc[data.非常规标签==col_name[2],"最终用途标签"] = col_name[4]
            # 3、词语规则匹配
            else:
                guize =  "|".join(list(filter(lambda x:str(x)!="nan",df[col_name])))  # 规则
                parten = re.compile(guize)
                print(guize,"\n")
                data.loc[(data.非常规标签.isnull())&
                         (data[col_name[1]].map(lambda x :re.search(parten,x)!=None)) &
                         (data.借贷标志.map(lambda x:x in col_name[0])) ,"非常规标签"]   = col_name[2]
                data.loc[data.非常规标签==col_name[2],"最终对手标签"] = col_name[3]
                data.loc[data.非常规标签==col_name[2],"最终用途标签"] = col_name[4]
        
        return data

In [37]:
data_lz_1 = biaoqian(data=data_lz,df=rule_ci)
data_lz_2 = biaoqian_2(data=data_lz_1)

('出账', '现金取出', '四合一')
取现|ATM取现|ATM取款|柜台取现|现金|卡取|本行ATM|卡取|网络ATM取款|现金支取|跨行自助取款|ATM取现 

('出账', '银行手续费支出', '四合一')
账户维护费|网银高级版|网银服务费|支付手续费|工本费|结算手续费|查询费|短信服务费|账户管理费|银联跨行手续费|跨行费|年费|户账管理费|自助设备使用费|收费项目:对公人民币转账、汇款（含退汇）-对公资金划转跨行同城|收费项目:对公人民币转账、汇款（含退汇）-对公资金划转跨行异地|资金信息综合服务-资金证明|账户维护-普通户账户管理费|资金信息综合服务-资金证明|对公自助服务-自助设备使用费|开换卡费|信使费|挂失费|汇费|异地费|银联手续费|支付手续费|银行手续费|凭证手续费 

('出账', '理财产品转出', '四合一')
理财产品扣划|理财产品扣款|购买理财产品|申购理财产品|购买理财|基金认购 

('出账', '运营支出', '四合一')
加班费|餐费|团队建设费|办公用品费|维修费|运费|住宿费|设备费|域名|半年奖|年终奖|采访费|招聘费|制作费|拓展费|印名片费|名片费|认证费|水费|电费|SWDS|SWGS|印刷费|协会会费|代扣税|评估费|咨询费|装修费|论坛费|清洗费|清洁费|茶水费|社保局代收付系统|社保代收|装饰材料|考察费|审计费|推广|宣传|广告|硬广|工资|季度奖|提成|奖金|差旅费|费用|报销|水电费|租金|房租|管理费|租赁费|租赁款|交通费|服务费|设备|维护费|培训费|电脑|文具|物管费|服务器|复印|招待费|经费|活动|团建费|采购|物业费|见面会|合作费|水果费|出差|电信费|通讯费|网络费|福利费|材料费|资料费|软件款|员工工装|购买前台系统|装修款|国.{0,5}税|订购 

('出账', '消费支出', '四合一')
手机充值|微信充值|微信红包|酒店|宾馆|服饰|服装|保险费|首都机场商贸|同程旅游|摩拜单车|购物中心|路易威登|乔治阿玛尼|餐饮|专卖店|医院|携程|微信支付|华为钱包|超市|加油站|旅游|旅行社|购物中心|火锅|烧烤|玩具店|华润万家|蛋糕|购物广场|购物休闲广场|银联消费|票务|百货|商店|男装|女装|机场客运|饭店|中移电商|苏宁电器|淘宝|支付宝外部商户|天猫|国美电

('出账', '公募基金转出', '对方户名')
上海东方证券资产管理有限公司|华融证券股份有限公司|山西证券股份有限公司|国都证券有限责任公司|浙江浙商证券资产管理有限公司|渤海汇金证券资产管理有限公司|东兴证券股份有限公司|泰康资产管理有限责任公司|北京高华证券有限责任公司|中银国际证券有限公司|财通证券资产管理有限公司|长江证券（上海）资产管理有限公司|华泰证券（上海）资产管理有限公司|中国人保资产管理股份有限公司|中泰证券（上海）资产管理有限公司|基金管理 

('出账', '消费支出', '对方户名')
中国铁路总公司资金清算中心|东北电力有限公司|中国平安财产保险股份有限公司|陕西首信商贸有限责任公司|乔治阿玛尼（上海）商贸有限公司|大商哈尔滨新一百购物广场有限公司|敖麓谷雅投资有限公司|北京京东叁佰陆拾度电子商务有限公司|苏州创旅天下信息技术有限公司|苏州同程艺龙网络科技有限公司|哈尔滨卓展时代广场百货有限公司|路易威登（中国）商业销售有限公司|中兴-沈阳商业大厦(集团)股份有限公司|合肥飞友网络科技有限公司|深圳市年年卡网络科技有限公司|中国南方航空股份有限公司|上海华程西南国际旅行社有限公司|中国太平洋人寿保险股份有限公司|辽宁有线电视网络传输有限公司|好利来（中国）电子科技股份有限公司|中国石油天然气股份有限公司|中国石化股份有限公司|铁道部资金清算中心|中华联合财产保险股份有限公司|武汉天河机场空港服务有限公司|上海华程西南旅行社有限公司|中移电子商务有限公司|苏宁电器集团有限公司|苏宁易购集团股份有限公司|支付宝外部商户|沈阳苏宁云商销售有限公司|中国移动通信|中国联通通信 

('出账', '运营支出', '对方户名')
今日头条有限公司|阿里云计算有限公司|保全世纪（北京）保安服务有限公司|北京昂然时代广告有限公司|北京二六三企业通信有限公司|北京基调网络股份有限公司|北京集奥聚合科技有限公司|北京拉勾网络技术有限公司|北京启明星辰信息安全技术有限公司|北京容联易通信息技术有限公司|北京融城教育咨询有限公司|北京神州微融金融信息服务有限公司|北京天威诚信电子商务服务有限公司|北京天信博易科技有限公司|北京轩辕广告有限公司|北京亿玛在线科技有限公司|北京亿美软通科技有限公司|北京银讯财富信息技术有限公司|北京智云众网络科技有限公司|北京众慧

In [38]:
# 非常规统计
data_lz_2.非常规标签.value_counts()

疑似返利款    2161
疑似投资款    1707
返利款       264
投资款        99
购房款        30
购车款        25
营业收入       16
运营支出       13
Name: 非常规标签, dtype: int64

## 工商关系

In [39]:
########
data_gs = pd.read_excel(r"策略文件\工商关联.xlsx",sheet_name="工商关系",dtype=np.str)
data_gs

,企业名称,工商关联
0,六安金色海港餐饮有限公司,关联企业
1,海南金色新天地庄园养老服务有限公司,关联企业
2,三亚瑞彩海景酒店有限公司大东海分公司,关联企业
3,安徽金色新天地庄园养老服务有限公司,关联企业
4,安徽霍山县伯爵商务酒店有限公司温泉假日酒店分公司,关联企业
5,六安金色新天地餐饮有限公司,关联企业
6,安徽伯爵庄园养老服务有限公司,关联企业
7,安徽霍山县伯爵商务酒店有限公司农业生态旅游度假村分公司,关联企业
8,三亚瑞彩海景酒店有限公司,关联企业
9,安徽霍山县伯爵温泉假日酒店有限公司,关联企业


In [40]:
x_o = data_lz_2.merge(data_gs[["企业名称","工商关联"]],how="left",left_on="对方户名",right_on="企业名称")
x_o.drop(columns=["企业名称"],inplace = True)

### 最终标签
#### 最终对手，最终用途补充

In [41]:
# 先工商，再非常规，再字面
def z_ds(x,y,z):
    # x 字面
    # y 非常规，对应身份用途
    # z 工商
    if str(z)!= "nan":
        xyz = z
    elif str(y) != "nan":
        xyz = y
    else:
        xyz = x
    return xyz
    
x_o["最终对手标签"] = list(map(lambda x,y,z:z_ds(x,y,z),x_o.字面用途标签,x_o.最终对手标签,x_o.工商关联))
x_o["最终用途标签"] = list(map(lambda x,y,z:z_ds(x,y,z),x_o.字面用途标签,x_o.最终用途标签,x_o.工商关联))

In [42]:
# 最终修正
# 1 名下其他
x_o.loc[x_o.交易户名 == x_o.对方户名,"最终对手标签"] = "名下其他账户"
x_o.loc[(x_o.借贷标志=="进")&(x_o.交易户名 == x_o.对方户名),"最终用途标签"] = "名下其他账户转入"
x_o.loc[(x_o.借贷标志=="出")&(x_o.交易户名 == x_o.对方户名),"最终用途标签"] = "名下其他账户转出"
# 2 关联企业，关联人员
## 身份不做，做用途
x_o.loc[(x_o.最终对手标签=="关联企业")&(x_o.借贷标志=="进"),"最终用途标签"] = "关联企业转入"
x_o.loc[(x_o.最终对手标签=="关联企业")&(x_o.借贷标志=="出"),"最终用途标签"] = "关联企业转出"
x_o.loc[(x_o.最终对手标签=="关联人员")&(x_o.借贷标志=="进"),"最终用途标签"] = "关联人员转入"
x_o.loc[(x_o.最终对手标签=="关联人员")&(x_o.借贷标志=="出"),"最终用途标签"] = "关联人员转出"

# 4 银行利息，手续费
x_o.loc[(x_o.最终对手标签=="银行利息"),"最终用途标签"] = "银行利息收入"
x_o.loc[(x_o.最终用途标签=="银行手续费支出"),"最终对手标签"] = "银行手续费"

In [43]:
x_o

,交易账号,交易卡号,交易户名,交易日期,交易时间,交易币种,借贷标志,交易金额,交易余额,交易类型,...,摘要,备注,附言,四合一,有误账款标记,字面用途标签,非常规标签,最终对手标签,最终用途标签,工商关联
0,125012117920000,6214835640073365,揭瑞,2016-04-29,09:01:23,CNY,进,20.00,20.00,金融交易,...,,,NaN,金融交易___nan,NaN,NaN,NaN,NaN,NaN,NaN
1,125012117920000,6214835640073365,揭瑞,2016-04-29,09:03:52,CNY,出,20.00,0.00,金融交易,...,专业版移动证书工本费（经典版）,,NaN,金融交易_专业版移动证书工本费（经典版）__nan,NaN,银行手续费支出,NaN,银行手续费,银行手续费支出,NaN
2,125012117920000,6214835640073365,揭瑞,2016-04-29,09:19:12,CNY,进,50000.00,50000.00,金融交易,...,银联卡转入,,NaN,金融交易_银联卡转入__nan,NaN,NaN,NaN,NaN,NaN,NaN
3,125012117920000,6214835640073365,揭瑞,2016-05-01,19:21:38,CNY,出,5000.00,45000.00,金融交易,...,,,NaN,金融交易___nan,NaN,NaN,NaN,NaN,NaN,NaN
4,125012117920000,6214835640073365,揭瑞,2016-05-01,19:21:39,CNY,进,5000.00,50000.00,金融交易,...,冲账：20160501流水号：K3581900176031A,,NaN,金融交易_冲账：20160501流水号：K3581900176031A__nan,NaN,NaN,NaN,NaN,NaN,NaN
5,125012117920000,6214835640073365,揭瑞,2016-05-01,19:22:50,CNY,出,5000.00,45000.00,金融交易,...,,,NaN,金融交易___nan,NaN,NaN,NaN,NaN,NaN,NaN
6,125012117920000,6214835640073365,揭瑞,2016-05-01,19:22:52,CNY,进,5000.00,50000.00,金融交易,...,冲账：20160501流水号：K3639500156655A,,NaN,金融交易_冲账：20160501流水号：K3639500156655A__nan,NaN,NaN,NaN,NaN,NaN,NaN
7,125012117920000,6214835640073365,揭瑞,2016-05-01,19:25:12,CNY,出,5000.00,45000.00,金融交易,...,,,NaN,金融交易___nan,NaN,NaN,NaN,NaN,NaN,NaN
8,125012117920000,6214835640073365,揭瑞,2016-05-01,19:25:14,CNY,进,5000.00,50000.00,金融交易,...,冲账：20160501流水号：K4989600195803A,,NaN,金融交易_冲账：20160501流水号：K4989600195803A__nan,NaN,NaN,NaN,NaN,NaN,NaN
9,125012117920000,6214835640073365,揭瑞,2016-05-01,19:34:15,CNY,出,10000.00,40000.00,金融交易,...,,,NaN,金融交易___nan,NaN,NaN,疑似返利款,名下其他账户,名下其他账户转出,关联人员


In [49]:
x_o.drop(columns=["四合一","有误账款标记"],inplace=True)

In [50]:
# x_o 资金结果写出
x_o.to_excel(r"策略文件\资金结果.xlsx",index=False)
# tj_3 基础统计结果写出
tj_3.to_excel(r"策略文件\统计结果.xlsx",index=False)

In [57]:
tj_3 = pd.DataFrame({"a":["askjj"]})
tj_3.to_excel(r"策略文件\结果\统计结果.xlsx",index=False)

FileNotFoundError: [Errno 2] No such file or directory: '策略文件\\结果\\统计结果.xlsx'